In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import pylab as pl
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib as plt
%matplotlib inline

In [2]:
gene = pd.read_excel('gene_expression_table4.xlsx')
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]
print(just_expression)
just_expression.as_matrix  # transformed dataframe with expression values to array
just_expression.shape
dist_matrix= dist.pdist(just_expression)
dist_square_matrix = dist.squareform(dist_matrix)

          5       2       6        4        3       1     1.1     5.1     3.1  \
0    3373.5  3757.8  3380.3   4560.2   4682.9  4182.9  5050.6  5820.5  4527.5   
1    1383.0  1501.8  1599.8   1433.7   1359.0  1565.5  1264.8  1285.5  1228.7   
2     864.7  1184.6  1465.4   1387.8   1414.6  1589.6  1847.7  2322.5  2966.6   
3     721.8   940.9   907.6    975.7    865.1   803.3  1417.6   995.4  1650.5   
4     419.0   389.1   390.7    356.1    452.8   400.2   866.8   580.3   637.7   
5     575.4   439.0   496.2    549.3    667.7   588.9   741.2   661.5   821.5   
6      85.9   191.7   113.9     67.4     71.8    89.9   219.8   256.2   267.4   
7     959.6  1083.6  1162.8   1130.0   1088.6  1326.4  1484.5  1677.5  1838.5   
8    1273.6  1216.4  1254.3   1150.1   1242.5  1153.8  1424.4  1736.1  1548.4   
9    4634.5  3749.6  4443.9   3844.0   4063.3  3896.4  3717.0  3725.6  3466.5   
10    327.7   385.4   477.3    259.9    485.8   315.3   608.6   788.1   692.1   
11   4159.3  4099.5  3874.2 

# Difference of Means

In [3]:
BRCA1_plus= just_expression.iloc[:,0:6]

BRCA1_minus= just_expression.iloc[:,7:12]

BRCA1_plus_means=np.mean(BRCA1_plus.T)

BRCA1_minus_means=np.mean(BRCA1_minus.T)

concatenated_means= np.c_[BRCA1_plus_means,BRCA1_minus_means]

expression_change_abs= np.abs(np.diff(concatenated_means))

expression_change_raw= np.diff(concatenated_means)

concatenated_means=np.append(expression_change_abs,expression_change_raw,1) # appended raw and abs value of means

In [4]:
labels=gene.iloc[0:373,0:3] # isolated gene names and functions
names=['mean','raw mean']
mean_pandas2= pd.DataFrame(concatenated_means,columns=names) # transformed concatenated means to dataframe

mean_append= np.append(labels,mean_pandas2,1) # appended gene labels to concatenated means

names2= ['gene name', 'symbol','function','abs value mean difference','raw mean difference']
labels_and_means =pd.DataFrame(mean_append,columns=names2)

sorted_means=labels_and_means.sort(['abs value mean difference'],ascending=False)
#print(sorted_means)
selected_top_20 = sorted_means.iloc[0:20,]
selected_top_20

/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,gene name,symbol,function,abs value mean difference,raw mean difference
33,J00105,B2M,MHC,5113.19,5113.19
89,U78525,EIF3S9,translation,4419.59,-4419.59
193,U71088,MAP2K5,signal transduction,3964.28,-3964.28
42,M19311,CALM1,cell cycle,3690.3,3690.3
153,M97796,ID2,transcription,3498.07,3498.07
63,M20471,CLTA,structure,3286.91,-3286.91
302,U53347,SLC1A5,transport,3277.22,-3277.22
318,M34338,SRM,synthase,2935.18,-2935.18
138,D49824,HLAB-null,receptor,2921.48,2921.48
242,Y12711,PBP,receptor,2904.28,2904.28


In [5]:
#from pandas import *
#idx = Int64Index([np.arange(373)])9
#index_sorted_means = DataFrame(index = np.arange(sorted_means.size), data =(sorted_means))
#index_sorted_means
#sorted_means
names_labels=['labels']
index_sorted_means = pd.DataFrame(np.arange(373),columns=names_labels)

data_names= ['gene order (high difference to low)','gene name', 'symbol','function','abs value mean difference','raw mean difference']
a = np.append(index_sorted_means, sorted_means,1)
my_labels= pd.DataFrame(a, columns=data_names)
my_labels2= my_labels[my_labels.function !='unknown']
my_labels2_select = my_labels2.loc[0:20,]
my_labels2_select

,gene order (high difference to low),gene name,symbol,function,abs value mean difference,raw mean difference
0,0,J00105,B2M,MHC,5113.19,5113.19
1,1,U78525,EIF3S9,translation,4419.59,-4419.59
2,2,U71088,MAP2K5,signal transduction,3964.28,-3964.28
3,3,M19311,CALM1,cell cycle,3690.3,3690.3
4,4,M97796,ID2,transcription,3498.07,3498.07
5,5,M20471,CLTA,structure,3286.91,-3286.91
6,6,U53347,SLC1A5,transport,3277.22,-3277.22
7,7,M34338,SRM,synthase,2935.18,-2935.18
8,8,D49824,HLAB-null,receptor,2921.48,2921.48
9,9,Y12711,PBP,receptor,2904.28,2904.28


In [ ]:
from ggplot import *
limit_graph = my_labels2.iloc[0:373, :]
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'abs value mean difference', color = 'function')) + \
    geom_point(size = 45) + \
    ggtitle("Difference of Means For All 373 Genes")

In [ ]:
from ggplot import *
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'raw mean difference', color = 'function')) + \
    geom_point(size = 100) + \
    ggtitle("Difference of Mean Gene Expression For All Genes (BRCA high - BRCA low)", ) 

In [ ]:
from ggplot import *
limit_graph = my_labels2.iloc[0:20, :]
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'abs value mean difference', color = 'function')) + \
    geom_point(size = 45) + \
    ggtitle("Difference of Means For Top 20 Genes")

In [ ]:
from ggplot import *
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'raw mean difference', color = 'function')) + \
    geom_point(size = 100) + \
    ggtitle("Difference of Mean Gene Expression For Top 20 (BRCA high - BRCA low)", ) 